# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

首先，我们可以写一个普通的python代码来完成这个小游戏：

In [1]:
def fizz_buzz_encoder(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else: return 0

def fizz_buzz_decoder(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]
   
for i in range(1,16):
     print(fizz_buzz_decoder(i, fizz_buzz_encoder(i)))

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz


接下来，我们尝试使用PyTorch构建一个简单的神经网络模型来玩这个小游戏。
## 1.准备训练数据

In [2]:
import numpy as np
import torch

NUM_DIGITS = 10

# 用二进制numpy数组表示输入数字
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encoder(i) for i in range(101, 2 ** NUM_DIGITS)]) # 由于我们是4分类问题，所以Y需要为整型数据

## 2.用PyTorch定义模型

In [3]:
# 隐藏层的维度设置为100
NUM_HIDDEN = 100

model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

# 使用cuda加速
if torch.cuda.is_available():
    model = model.cuda()

## 3.定义损失函数和优化函数

In [4]:
# 由于是分类问题，我们选择交叉熵函数
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

## 4.开始训练

In [5]:
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]
        # 将数据放到cuda上
        if torch.cuda.is_available():
            batchX = batchX.cuda()
            batchY = batchY.cuda()
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 打印loss
    print('Epoch:', epoch, 'Loss:', loss.item())

Epoch: 0 Loss: 1.1479040384292603
Epoch: 1 Loss: 1.1204196214675903
Epoch: 2 Loss: 1.1173627376556396
Epoch: 3 Loss: 1.116934895515442
Epoch: 4 Loss: 1.116753101348877
Epoch: 5 Loss: 1.1164082288742065
Epoch: 6 Loss: 1.1158692836761475
Epoch: 7 Loss: 1.1151834726333618
Epoch: 8 Loss: 1.1143817901611328
Epoch: 9 Loss: 1.1135069131851196
Epoch: 10 Loss: 1.1125788688659668
Epoch: 11 Loss: 1.1116251945495605
Epoch: 12 Loss: 1.1106680631637573
Epoch: 13 Loss: 1.1097053289413452
Epoch: 14 Loss: 1.108749270439148
Epoch: 15 Loss: 1.1078239679336548
Epoch: 16 Loss: 1.1069161891937256
Epoch: 17 Loss: 1.106016993522644
Epoch: 18 Loss: 1.1051347255706787
Epoch: 19 Loss: 1.1042708158493042
Epoch: 20 Loss: 1.1034204959869385
Epoch: 21 Loss: 1.1025844812393188
Epoch: 22 Loss: 1.1017569303512573
Epoch: 23 Loss: 1.1009509563446045
Epoch: 24 Loss: 1.1001569032669067
Epoch: 25 Loss: 1.099371314048767
Epoch: 26 Loss: 1.0985989570617676
Epoch: 27 Loss: 1.0978518724441528
Epoch: 28 Loss: 1.0971248149871826


Epoch: 459 Loss: 0.8908032178878784
Epoch: 460 Loss: 0.8889681696891785
Epoch: 461 Loss: 0.8892804384231567
Epoch: 462 Loss: 0.8882343173027039
Epoch: 463 Loss: 0.8875031471252441
Epoch: 464 Loss: 0.8868914246559143
Epoch: 465 Loss: 0.8857057094573975
Epoch: 466 Loss: 0.8853287100791931
Epoch: 467 Loss: 0.8850395083427429
Epoch: 468 Loss: 0.8839953541755676
Epoch: 469 Loss: 0.8837028741836548
Epoch: 470 Loss: 0.8830230236053467
Epoch: 471 Loss: 0.8817740678787231
Epoch: 472 Loss: 0.881100058555603
Epoch: 473 Loss: 0.8812659978866577
Epoch: 474 Loss: 0.8797308802604675
Epoch: 475 Loss: 0.8786056041717529
Epoch: 476 Loss: 0.8771948218345642
Epoch: 477 Loss: 0.8771919012069702
Epoch: 478 Loss: 0.8769286274909973
Epoch: 479 Loss: 0.8761181831359863
Epoch: 480 Loss: 0.8753253817558289
Epoch: 481 Loss: 0.8750395774841309
Epoch: 482 Loss: 0.8740843534469604
Epoch: 483 Loss: 0.8728519678115845
Epoch: 484 Loss: 0.8731392025947571
Epoch: 485 Loss: 0.8738431334495544
Epoch: 486 Loss: 0.8714079856

Epoch: 914 Loss: 0.4216865003108978
Epoch: 915 Loss: 0.4201201796531677
Epoch: 916 Loss: 0.42140305042266846
Epoch: 917 Loss: 0.4193773567676544
Epoch: 918 Loss: 0.41647014021873474
Epoch: 919 Loss: 0.41701340675354004
Epoch: 920 Loss: 0.4175388514995575
Epoch: 921 Loss: 0.41455838084220886
Epoch: 922 Loss: 0.41488325595855713
Epoch: 923 Loss: 0.41266417503356934
Epoch: 924 Loss: 0.41221681237220764
Epoch: 925 Loss: 0.411172091960907
Epoch: 926 Loss: 0.40913620591163635
Epoch: 927 Loss: 0.40945637226104736
Epoch: 928 Loss: 0.40640443563461304
Epoch: 929 Loss: 0.4074641466140747
Epoch: 930 Loss: 0.40553897619247437
Epoch: 931 Loss: 0.405029296875
Epoch: 932 Loss: 0.40431880950927734
Epoch: 933 Loss: 0.402466744184494
Epoch: 934 Loss: 0.40189287066459656
Epoch: 935 Loss: 0.4009256362915039
Epoch: 936 Loss: 0.39968571066856384
Epoch: 937 Loss: 0.3977562189102173
Epoch: 938 Loss: 0.3975355327129364
Epoch: 939 Loss: 0.39755818247795105
Epoch: 940 Loss: 0.39565369486808777
Epoch: 941 Loss: 0

Epoch: 1352 Loss: 0.13875530660152435
Epoch: 1353 Loss: 0.1387852132320404
Epoch: 1354 Loss: 0.13833287358283997
Epoch: 1355 Loss: 0.13857977092266083
Epoch: 1356 Loss: 0.13757123053073883
Epoch: 1357 Loss: 0.13763059675693512
Epoch: 1358 Loss: 0.13762271404266357
Epoch: 1359 Loss: 0.13662102818489075
Epoch: 1360 Loss: 0.13652756810188293
Epoch: 1361 Loss: 0.1368204802274704
Epoch: 1362 Loss: 0.13595572113990784
Epoch: 1363 Loss: 0.1359712779521942
Epoch: 1364 Loss: 0.13489653170108795
Epoch: 1365 Loss: 0.13575589656829834
Epoch: 1366 Loss: 0.13454383611679077
Epoch: 1367 Loss: 0.13443517684936523
Epoch: 1368 Loss: 0.13440029323101044
Epoch: 1369 Loss: 0.13382510840892792
Epoch: 1370 Loss: 0.13272416591644287
Epoch: 1371 Loss: 0.13364261388778687
Epoch: 1372 Loss: 0.1324571967124939
Epoch: 1373 Loss: 0.13321606814861298
Epoch: 1374 Loss: 0.13227833807468414
Epoch: 1375 Loss: 0.13173271715641022
Epoch: 1376 Loss: 0.13176047801971436
Epoch: 1377 Loss: 0.13149049878120422
Epoch: 1378 Loss

Epoch: 1784 Loss: 0.06452049314975739
Epoch: 1785 Loss: 0.0629153698682785
Epoch: 1786 Loss: 0.0642639771103859
Epoch: 1787 Loss: 0.06387148052453995
Epoch: 1788 Loss: 0.06296459585428238
Epoch: 1789 Loss: 0.064503513276577
Epoch: 1790 Loss: 0.06384739279747009
Epoch: 1791 Loss: 0.06419723480939865
Epoch: 1792 Loss: 0.06208627298474312
Epoch: 1793 Loss: 0.06341852992773056
Epoch: 1794 Loss: 0.06286808848381042
Epoch: 1795 Loss: 0.06266213953495026
Epoch: 1796 Loss: 0.0628523901104927
Epoch: 1797 Loss: 0.061686452478170395
Epoch: 1798 Loss: 0.06388834118843079
Epoch: 1799 Loss: 0.06209755688905716
Epoch: 1800 Loss: 0.06197686493396759
Epoch: 1801 Loss: 0.06199236214160919
Epoch: 1802 Loss: 0.061048250645399094
Epoch: 1803 Loss: 0.06278029084205627
Epoch: 1804 Loss: 0.06111226975917816
Epoch: 1805 Loss: 0.06224408745765686
Epoch: 1806 Loss: 0.06233895197510719
Epoch: 1807 Loss: 0.06108997389674187
Epoch: 1808 Loss: 0.06159651651978493
Epoch: 1809 Loss: 0.062103476375341415
Epoch: 1810 Lo

...

Epoch: 8075 Loss: 0.002665713895112276
Epoch: 8076 Loss: 0.0026679039001464844
Epoch: 8077 Loss: 0.0026697407010942698
Epoch: 8078 Loss: 0.0026647073682397604
Epoch: 8079 Loss: 0.00266433646902442
Epoch: 8080 Loss: 0.0026667912025004625
Epoch: 8081 Loss: 0.0026646896731108427
Epoch: 8082 Loss: 0.0026615108363330364
Epoch: 8083 Loss: 0.0026634004898369312
Epoch: 8084 Loss: 0.002657748991623521
Epoch: 8085 Loss: 0.002663841936737299
Epoch: 8086 Loss: 0.002662040526047349
Epoch: 8087 Loss: 0.0026528569869697094
Epoch: 8088 Loss: 0.0026664556935429573
Epoch: 8089 Loss: 0.0026624996680766344
Epoch: 8090 Loss: 0.0026629942003637552
Epoch: 8091 Loss: 0.002659938996657729
Epoch: 8092 Loss: 0.002660769037902355
Epoch: 8093 Loss: 0.002662376035004854
Epoch: 8094 Loss: 0.0026598682161420584
Epoch: 8095 Loss: 0.0026596917305141687
Epoch: 8096 Loss: 0.002658137585967779
Epoch: 8097 Loss: 0.002658367156982422
Epoch: 8098 Loss: 0.002655029296875
Epoch: 8099 Loss: 0.0026526274159550667
Epoch: 8100 Los

Epoch: 8489 Loss: 0.0024680031929165125
Epoch: 8490 Loss: 0.002470670035108924
Epoch: 8491 Loss: 0.002467420417815447
Epoch: 8492 Loss: 0.0024683387018740177
Epoch: 8493 Loss: 0.0024662900250405073
Epoch: 8494 Loss: 0.002466449048370123
Epoch: 8495 Loss: 0.00246459455229342
Epoch: 8496 Loss: 0.002464859513565898
Epoch: 8497 Loss: 0.002463605720549822
Epoch: 8498 Loss: 0.0024650362320244312
Epoch: 8499 Loss: 0.002460532821714878
Epoch: 8500 Loss: 0.0024593141861259937
Epoch: 8501 Loss: 0.002465212717652321
Epoch: 8502 Loss: 0.002455216832458973
Epoch: 8503 Loss: 0.0024627225939184427
Epoch: 8504 Loss: 0.0024595260620117188
Epoch: 8505 Loss: 0.0024614864960312843
Epoch: 8506 Loss: 0.0024567004293203354
Epoch: 8507 Loss: 0.002460356103256345
Epoch: 8508 Loss: 0.0024588550440967083
Epoch: 8509 Loss: 0.0024648949038237333
Epoch: 8510 Loss: 0.0024550226517021656
Epoch: 8511 Loss: 0.0024595437571406364
Epoch: 8512 Loss: 0.0024596850853413343
Epoch: 8513 Loss: 0.0024544398766011
Epoch: 8514 Lo

Epoch: 8903 Loss: 0.0022903901990503073
Epoch: 8904 Loss: 0.002289436524733901
Epoch: 8905 Loss: 0.0022880060132592916
Epoch: 8906 Loss: 0.002284173620864749
Epoch: 8907 Loss: 0.0022860809694975615
Epoch: 8908 Loss: 0.002292368095368147
Epoch: 8909 Loss: 0.002288677031174302
Epoch: 8910 Loss: 0.0022863636258989573
Epoch: 8911 Loss: 0.0022862928453832865
Epoch: 8912 Loss: 0.0022901957854628563
Epoch: 8913 Loss: 0.0022878823801875114
Epoch: 8914 Loss: 0.0022860278841108084
Epoch: 8915 Loss: 0.0022843678016215563
Epoch: 8916 Loss: 0.0022848269436508417
Epoch: 8917 Loss: 0.0022807475179433823
Epoch: 8918 Loss: 0.002283608540892601
Epoch: 8919 Loss: 0.002284862333908677
Epoch: 8920 Loss: 0.0022837850265204906
Epoch: 8921 Loss: 0.002282902132719755
Epoch: 8922 Loss: 0.0022819307632744312
Epoch: 8923 Loss: 0.002282725414261222
Epoch: 8924 Loss: 0.0022773919627070427
Epoch: 8925 Loss: 0.002280500251799822
Epoch: 8926 Loss: 0.0022792110685259104
Epoch: 8927 Loss: 0.0022821426391601562
Epoch: 89

Epoch: 9317 Loss: 0.0021316034253686666
Epoch: 9318 Loss: 0.0021319035440683365
Epoch: 9319 Loss: 0.002127559157088399
Epoch: 9320 Loss: 0.0021314620971679688
Epoch: 9321 Loss: 0.0021299785003066063
Epoch: 9322 Loss: 0.0021310735028237104
Epoch: 9323 Loss: 0.0021308085415512323
Epoch: 9324 Loss: 0.0021260755602270365
Epoch: 9325 Loss: 0.0021262168884277344
Epoch: 9326 Loss: 0.0021290425211191177
Epoch: 9327 Loss: 0.0021286187693476677
Epoch: 9328 Loss: 0.0021263405214995146
Epoch: 9329 Loss: 0.002125086495652795
Epoch: 9330 Loss: 0.0021245745010674
Epoch: 9331 Loss: 0.0021293957252055407
Epoch: 9332 Loss: 0.00212830095551908
Epoch: 9333 Loss: 0.0021293957252055407
Epoch: 9334 Loss: 0.0021203889045864344
Epoch: 9335 Loss: 0.002126305131241679
Epoch: 9336 Loss: 0.0021203889045864344
Epoch: 9337 Loss: 0.002123868092894554
Epoch: 9338 Loss: 0.002122702309861779
Epoch: 9339 Loss: 0.0021222431678324938
Epoch: 9340 Loss: 0.002123461803421378
Epoch: 9341 Loss: 0.0021221020724624395
Epoch: 9342

Epoch: 9731 Loss: 0.0019944331143051386
Epoch: 9732 Loss: 0.0019909541588276625
Epoch: 9733 Loss: 0.0019916605670005083
Epoch: 9734 Loss: 0.0019901946652680635
Epoch: 9735 Loss: 0.001990795135498047
Epoch: 9736 Loss: 0.0019894177094101906
Epoch: 9737 Loss: 0.00199139560572803
Epoch: 9738 Loss: 0.0019850025419145823
Epoch: 9739 Loss: 0.0019890645053237677
Epoch: 9740 Loss: 0.001986274030059576
Epoch: 9741 Loss: 0.001987174618989229
Epoch: 9742 Loss: 0.0019861680921167135
Epoch: 9743 Loss: 0.001986697781831026
Epoch: 9744 Loss: 0.0019842961337417364
Epoch: 9745 Loss: 0.0019872805569320917
Epoch: 9746 Loss: 0.0019848435185849667
Epoch: 9747 Loss: 0.001986362272873521
Epoch: 9748 Loss: 0.001988569973036647
Epoch: 9749 Loss: 0.001983359921723604
Epoch: 9750 Loss: 0.0019863094203174114
Epoch: 9751 Loss: 0.0019826889038085938
Epoch: 9752 Loss: 0.0019840840250253677
Epoch: 9753 Loss: 0.001983289374038577
Epoch: 9754 Loss: 0.0019835366401821375
Epoch: 9755 Loss: 0.0019785563927143812
Epoch: 975

## 5.准备测试数据

In [6]:
# 我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])

if torch.cuda.is_available():
    testX = testX.cuda()

## 6.测试

In [7]:
# 由于测试时不需要保留梯度，故把测试时的梯度清空
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))
print([fizz_buzz_decoder(i, x) for (i, x) in predictions])

testY = testY.cpu()
print("accuracy:", np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encoder(i) for i in range(1,101)])), "%")

['1', 'buzz', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', 'fizz', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', 'buzz', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', 'fizz', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']
accuracy: 96 %
